In [1]:
from __future__ import annotations

from pathlib import Path
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

import photon_tools as pt

In [2]:
# Beispiel: alle HDF5 in einem Ordner
root = Path("/Users/kappel/Diss/20-29_raw_data/20_all/24_04_16/77K-10nM-AF488-RM-mu30-vitrified-IR-02/05uW/timestamps/")

paths = sorted(root.rglob("*.h5"))  # oder "*.h5"
# paths = sorted(root.glob("*.hdf5"))

len(paths), paths[0] if paths else None

(34,
 PosixPath('/Users/kappel/Diss/20-29_raw_data/20_all/24_04_16/77K-10nM-AF488-RM-mu30-vitrified-IR-02/05uW/timestamps/tt_AF488_10nM_RM_mu30_00mM_salt_05uW_77K_circpol_001.h5'))

In [3]:
results_path = Path("screening_results.csv")

if results_path.exists():
    df = pd.read_csv(results_path)
else:
    df = pd.DataFrame(columns=[
        "path", "keep", "note",
        "bin_width_ms", "channel",
        "timing_resolution",
    ])

# Für schnelles Lookup:
df_index = {p: i for i, p in enumerate(df["path"].astype(str).tolist())}
df.head()

,path,keep,note,bin_width_ms,channel,timing_resolution


In [ ]:
# Controls
idx_slider = widgets.IntSlider(
    value=0, min=0, max=max(len(paths)-1, 0), step=1,
    description="File",
    continuous_update=False,
    layout=widgets.Layout(width="600px")
)

btn_prev = widgets.Button(description="◀ Prev", layout=widgets.Layout(width="90px"))
btn_next = widgets.Button(description="Next ▶", layout=widgets.Layout(width="90px"))

bin_dd = widgets.Dropdown(
    options=[1, 2, 5, 10, 20, 50, 100],
    value=10,
    description="Bin (ms)",
    layout=widgets.Layout(width="220px")
)

channel_dd = widgets.Dropdown(
    options=[("all", "all")],
    value="all",
    description="Channel",
    layout=widgets.Layout(width="220px"),
)

timing_txt = widgets.FloatText(
    value=5e-9,
    description="timing (s)",
    layout=widgets.Layout(width="260px")
)

show_fn_chk = widgets.Checkbox(
    value=True, description="show filename in title",
    indent=False
)

keep_chk = widgets.Checkbox(
    value=False, description="KEEP",
    indent=False
)

note_txt = widgets.Textarea(
    value="",
    description="Note",
    layout=widgets.Layout(width="600px", height="90px")
)

btn_save = widgets.Button(description="💾 Save", button_style="success")
btn_save_next = widgets.Button(description="💾 Save & Next", button_style="success")

status = widgets.HTML(value="")

# Output area for plot
out = widgets.Output()

In [5]:
def _current_path() -> Path:
    if not paths:
        raise ValueError("No files found.")
    return paths[idx_slider.value]

def _load_existing_annotation(path: Path):
    """Load keep/note for this file if it exists in CSV."""
    s = str(path)
    if s in df_index:
        row = df.iloc[df_index[s]]
        keep_chk.value = bool(row["keep"])
        note_txt.value = "" if pd.isna(row["note"]) else str(row["note"])
    else:
        keep_chk.value = False
        note_txt.value = ""

def _render():
    """Load file -> preview -> display. Also refresh annotation widgets."""
    p = _current_path()
    _load_existing_annotation(p)

    with out:
        clear_output(wait=True)
        try:
            ds = pt.load(p, timing_resolution=float(timing_txt.value))
            update_channel_dropdown(ds)
            # preview returns fig; show=False so we control rendering here
            # map dropdown to preview() argument

            if ds.photons.detectors is None:
                channel_dd.options = [("all", "all")]
            else:
                chans = sorted(set(int(x) for x in ds.photons.detectors))
                channel_dd.options = [("all", "all")] + [(f"det {c}", c) for c in chans]
                
            channels = None if channel_dd.value == "all" else [int(channel_dd.value)]

            fig = pt.preview(
                ds,
                bin_width_ms=float(bin_dd.value),
                show_filename=bool(show_fn_chk.value),
                width=1000,
                height=420,
                show=False,
            )
            # Ensure scroll-zoom on final render
            fig.show(config={"scrollZoom": True, "displaylogo": False})
            status.value = f"<b>{idx_slider.value+1}/{len(paths)}</b> — {p}"
        except Exception as e:
            status.value = f"<span style='color:#b00'><b>Error:</b> {e}</span><br>{p}"


def update_channel_dropdown(ds):
    if ds.photons.detectors is None:
        # kein Detektor-Array → nur "all"
        channel_dd.options = [("all", "all")]
        channel_dd.value = "all"
        return

    # vorhandene Detektoren aus den Daten bestimmen
    dets = sorted(set(int(d) for d in ds.photons.detectors))

    # Dropdown-Optionen bauen
    opts = [("all", "all")] + [(f"det {d}", d) for d in dets]

    channel_dd.options = opts
    channel_dd.value = "all"
    

def _save():
    """Write/update the CSV row for current file."""
    global df, df_index

    p = _current_path()
    record = {
        "path": str(p),
        "keep": bool(keep_chk.value),
        "note": note_txt.value,
        "bin_width_ms": float(bin_dd.value),
        "timing_resolution": float(timing_txt.value),
    }

    s = str(p)
    if s in df_index:
        df.loc[df_index[s], :] = record
    else:
        df = pd.concat([df, pd.DataFrame([record])], ignore_index=True)
        df_index = {p: i for i, p in enumerate(df["path"].astype(str).tolist())}

    df.to_csv(results_path, index=False)
    status.value = status.value + " &nbsp;✅ saved"

In [6]:
def _clamp(i: int) -> int:
    return max(0, min(len(paths)-1, i))

def on_prev(_):
    idx_slider.value = _clamp(idx_slider.value - 1)

def on_next(_):
    idx_slider.value = _clamp(idx_slider.value + 1)

def on_save(_):
    _save()

def on_save_next(_):
    _save()
    on_next(_)

btn_prev.on_click(on_prev)
btn_next.on_click(on_next)
btn_save.on_click(on_save)
btn_save_next.on_click(on_save_next)

# Re-render when these change
for w in [idx_slider, bin_dd, channel_dd, timing_txt, show_fn_chk]:
    w.observe(lambda change: _render(), names="value")

In [7]:
controls_row1 = widgets.HBox([btn_prev, btn_next, idx_slider])
controls_row2 = widgets.HBox([bin_dd, timing_txt, show_fn_chk, keep_chk])
controls_row3 = widgets.HBox([btn_save, btn_save_next])

ui = widgets.VBox([
    controls_row1,
    controls_row2,
    note_txt,
    controls_row3,
    status,
    out,
])

display(ui)

# initial render
if paths:
    _render()
else:
    status.value = "<b>No files found.</b> Check your root/glob."